# SOX9 keeps growth plates and articular cartilage healthy by inhibiting chondrocyte dedifferentiation/osteoblastic redifferentiation
Haseeb *et al.* PNAS (2021)
 
 PMID: [33597301](https://pubmed.ncbi.nlm.nih.gov/33597301/)
 ***
 

## 0. Setting up workenvironment<a id="0"></a>

In [ ]:
suppressPackageStartupMessages({
    library(DropletUtils)
    library(SingleCellExperiment)
    library(scuttle)
    library(Seurat)
    library(SeuratWrappers)
    library(stringr)
    library(dplyr)
    library(data.table)
    library(Matrix)
    library(patchwork)
    library(ggplot2)
})

options(repr.plot.width = 16, repr.plot.height = 8)

## 1. Data analysis in Seurat

In [ ]:
seuratObject <- readRDS(file = "VSOX_Mutp13_QC.Rds")

In [ ]:
VlnPlot(seuratObject, features = c("nCount_RNA", "nFeature_RNA", "subsets_Mito_percent"), 
        ncol = 3, group.by = "orig.ident", pt.size = 0)

### 1.1. Feature selection

We next calculate a subset of features that exhibit high cell-to-cell variation in the dataset (i.e, they are highly expressed in some cells, and lowly expressed in others). We and others have found that focusing on these genes in downstream analysis helps to highlight biological signal in single-cell datasets.

The procedure in Seurat v3 directly models the mean-variance relationship inherent in single-cell data, and is implemented in the `FindVariableFeatures` function. By default, it defaults to 2,000 features (genes) per dataset. These will be used in downstream analysis, like principal component analysis (PCA).

In [ ]:
seuratObject <- NormalizeData(seuratObject, verbose = FALSE)
seuratObject <- FindVariableFeatures(seuratObject, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(seuratObject), 10)

In [ ]:
options(repr.plot.width=10)

# plot variable features with and without labels
plot1 <- VariableFeaturePlot(seuratObject)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot2

### 1.2. Scaling the data

Next, we apply a linear transformation (scaling) that is a standard pre-processing step prior to dimensional reduction techniques like PCA. The ScaleData function:

- Shifts the expression of each gene, so that the mean expression across cells is 0
- Scales the expression of each gene, so that the variance across cells is 1
- This step gives equal weight in downstream analyses, so that highly-expressed genes do not dominate. This is superior to log-normalized counts, which overemphasize the influence of small count fluctuations.
- The results of this are stored in `object[["originalexp"]]@scale.data`

In [ ]:
all.genes <- rownames(seuratObject)
seuratObject <- ScaleData(seuratObject, features = all.genes)

### 1.3 Linear dimensionality reduction: PCA

Next we perform PCA on the scaled data. By default, only the previously determined variable features are used as input, but can be defined using features argument if you wish to choose a different subset.

In [ ]:
seuratObject <- RunPCA(seuratObject, features = VariableFeatures(object = seuratObject), verbose = FALSE)

In [ ]:
DimPlot(seuratObject, reduction = "pca") + xlab("PC 1") + ylab("PC 2")

### 1.4. Determine dimensionality

To overcome the extensive technical noise in any single feature for scRNA-seq data, Seurat clusters cells based on their PCA scores, with each PC essentially representing a 'metafeature' that combines information across a correlated feature set. The top principal components therefore represent a robust compression of the dataset. However, how many componenets should we choose to include? 

[Macosko et al](http://www.cell.com/abstract/S0092-8674(15)00549-8) introduced a resampling test inspired by the JackStraw procedure. They randomly permute a subset of the data (1% by default) and rerun PCA, constructing a 'null distribution' of feature scores, and repeat this procedure. 'Significant' PCs are defined as those who have a strong enrichment of low p-value features.

**NOTE: This process can take a long time for big datasets, and is therefore commented out for expediency. More approximate techniques such as a scree plot implemented in `ElbowPlot()` can be used to reduce computation time.**

The `JackStrawPlot` function provides a visualization tool for comparing the distribution of p-values for each PC with a uniform distribution (dashed line). 'Significant' PCs will show a strong enrichment of features with low p-values (solid curve above the dashed line).

An alternative heuristic method generates an 'Elbow plot': a ranking of principle components based on the percentage of variance explained by each one (ElbowPlot function). 

In [ ]:
ElbowPlot(seuratObject, 50)

### 1.5. Clustering

Seurat v4 applies a graph-based clustering approach, building upon initial strategies in [Macosko et al](http://www.cell.com/abstract/S0092-8674(15)00549-8). Importantly, the distance metric which drives the clustering analysis (based on previously identified PCs) remains the same. The new method embeds cells in a graph structure - for example a K-nearest neighbor (KNN) graph, with edges drawn between cells with similar feature expression patterns, and then attempt to partition this graph into highly interconnected 'quasi-cliques' or 'communities'.

As in [PhenoGraph](https://doi.org/10.1016/j.cell.2015.05.047), Seurat first constructs a KNN graph based on the euclidean distance in PCA space, and refine the edge weights between any two cells based on the shared overlap in their local neighborhoods (Jaccard similarity). This step is performed using the `FindNeighbors` function, and takes as input the previously defined dimensionality of the dataset.

To cluster the cells, modularity optimization techniques such as the Louvain algorithm are then applied to iteratively group cells together, with the goal of optimizing the standard modularity function. The `FindClusters` function implements this procedure and contains a resolution parameter that sets the 'granularity' of the downstream clustering, with increased values leading to a greater number of clusters. Optimal resolution often increases for larger datasets.

In [ ]:
seuratObject <- FindNeighbors(seuratObject, dims = 1:25)
seuratObject <- FindClusters(seuratObject, resolution = 0.15)

### 1.6. Non-linear dimensionality reduction: UMAP

Seurat offers several non-linear dimensional reduction techniques, such as tSNE and UMAP, to visualize and explore these datasets. The goal of these algorithms is to learn the underlying manifold of the data in order to place similar cells together in low-dimensional space. Cells within the graph-based clusters determined above should co-localize on these dimension reduction plots. As input to the UMAP and tSNE, [it is recommended to use the same PCs](https://doi.org/10.1038/s41467-019-13056-x) as input to the clustering analysis.

In [ ]:
seuratObject <- RunUMAP(seuratObject, dims = 1:25, n.neighbors = 12, verbose = FALSE)

In [ ]:
options(repr.plot.width = 8)

DimPlot(seuratObject, reduction = "umap", label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

#ggsave("./6-kmita/images/E12_5_na.png" ,width=10, height=8)

### 1.7. Differential expression analysis
#### 1.7.1. Global annotation by differential expression analysis

Seurat identifies markers that define clusters via differential expression. By default, it identifes positive and negative markers of a single cluster (specified in ident.1), compared to all other cells. `FindAllMarkers` automates this process for all clusters, but it is possible to test groups of clusters vs. each other, or against all cells.

The `min.pct` argument requires a feature to be detected at a minimum percentage in either of the two groups of cells, and the `thresh.test` argument requires a feature to be differentially expressed (on average) by some amount between the two groups. You can set both of these to 0, but with a dramatic increase in time - since this will test a large number of features that are unlikely to be highly discriminatory. As another option to speed up these computations, `max.cells.per.ident` can be set. This will downsample each identity class to have no more cells than whatever this is set to. While there is generally going to be a loss in power, the speed increases can be significiant and the most highly differentially expressed features will likely still rise to the top.

In [ ]:
# Seurat default settings
seuratObject.markers <- FindAllMarkers(seuratObject, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

seuratObject.markers[seuratObject.markers$cluster == 3,] %>%
slice_max(n = 10
          , order_by = avg_log2FC)

In [ ]:
write.csv(seuratObject.markers, file = "markers.csv", quote = FALSE)

In [ ]:
#Early limb bud mesenchyme
options(repr.plot.width = 18, repr.plot.height = 8)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Meis1","Tbx4","Tbx5"), min.cutoff = "q1")  

p1 + p2

In [ ]:
#Limb bud mesenchyme
options(repr.plot.width = 18, repr.plot.height = 8)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Prrx1","Prrx2","Fgf10"), min.cutoff = "q1")  

p1 + p2

In [ ]:
#Interzone cells
options(repr.plot.width = 18, repr.plot.height = 8)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Smoc2","Dact2","Gdf5","Sox9"), min.cutoff = "q1", keep.scale = 'all')  

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 5)
FeaturePlot(seuratObject, features = c('Gdf5', 'Smoc2'), blend = TRUE )

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 5)
FeaturePlot(seuratObject, features = c('Gdf5', 'Col2a1'), blend = TRUE )

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 5)
FeaturePlot(seuratObject, features = c('Sox9', 'Smoc2'), blend = TRUE )

In [ ]:
#Joint progenitors
options(repr.plot.width = 18, repr.plot.height = 8)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Gdf5","Wnt9a","Smoc2","Sox9"), min.cutoff = "q1", order = TRUE,  keep.scale = 'all')  

p1 + p2

In [ ]:
#Muscle progenitors
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Pax7","Notch","Itm2a","Nrk"), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )  

p1 + p2

In [ ]:
#Myogenic stem cells
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Pax3","Pdgfc"), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )  

p1 + p2

In [ ]:
#Skeletal muscle cells
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Myhc","Tnnt1",'Myod','Tnnt3'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )  

p1 + p2

In [ ]:
#Smooth muscle cells
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Acta2"), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )  

p1 + p2

In [ ]:
#Cardiomyocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Foxp1"), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )  

p1 + p2

In [ ]:
#Tenocyte precursors
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Tnmd", "Prrx1", "Prrx2","Scx"), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )  

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 5)
FeaturePlot(seuratObject, features = c('Fgf8','Prrx1'), blend = TRUE )

In [ ]:
#Ectoderm
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Fgf8","Wnt7a","En1","Rspo2"), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )  

p1 + p2

In [ ]:
#VEC
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Pecam1","Flt1","Emcn"), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )  

p1 + p2

In [ ]:
#LEC
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Flt4","Prox1","Lyve1",'Flt1'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )  

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 5)
FeaturePlot(seuratObject, features = c('Flt4', 'Flt1'), blend = TRUE )

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Pf4","Car2","Cd68",'Gp9','Gypa'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Pf4","Cd3d","Cd68",'Mrc1','Camp','Siglech','Ptprc'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Ccl5",'Cd79b'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c('Gypa','Car2'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("","","",""), min.cutoff = "q1")  

p1 + p2

In [ ]:
DotPlot(object = seuratObject, features = c('','','',''), col.min = 0)

In [ ]:
Toptions(repr.plot.width = 18, repr.plot.height = 6)
FeaturePlot(A, features = c('', ''), blend = TRUE )

In [ ]:
# Step 1: Define the variable 'name' with cell type / global
name <- "Global"

# Step 2: Create the column name dynamically
column_name <- paste0("CellType_", name)

In [ ]:
seuratObject@meta.data[[column_name]] <- Idents(seuratObject)

In [ ]:
seuratObject@meta.data[[column_name]] <- as.character(seuratObject@meta.data[[column_name]])

seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '0'] <- 'Mes_lin'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '1'] <- 'Mes_lin'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '2'] <- 'Immune cells'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '3'] <- 'Immune cells'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '4'] <- 'Immune cells'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '5'] <- 'Immune cells'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '6'] <- 'Tenocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '7'] <- 'Erythroid cells'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '8'] <- 'Immune cells'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '9'] <- 'Mes_lin'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '10'] <- 'Vascular endothelial cells'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '11'] <- 'Immune cells'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '12'] <- 'Immune cells'

seuratObject@meta.data[[column_name]] <- as.factor(seuratObject@meta.data[[column_name]])

In [ ]:
options(repr.plot.width = 12)

DimPlot(seuratObject, reduction = "umap", label = TRUE, group.by = column_name) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
saveRDS(seuratObject, file = "VSOX_Mutp13_global.Rds")

## 2. Refinement of global annotations

### 2.1. Mesenchymal cells

In [ ]:
seuratObject <- readRDS("VSOX_Mutp13_global.Rds")

In [ ]:
Idents(seuratObject) <- seuratObject@meta.data$CellType_Global

In [ ]:
seuratObject <- subset(seuratObject, idents = c("Mes_lin"))

In [ ]:
seuratObject <- ScaleData(seuratObject)

In [ ]:
seuratObject <- RunPCA(seuratObject, features = VariableFeatures(object = seuratObject), verbose = FALSE)

In [ ]:
ElbowPlot(seuratObject, 50)

In [ ]:
seuratObject <- FindNeighbors(seuratObject, dims = 1:25, verbose = FALSE)
seuratObject <- FindClusters(seuratObject, resolution = 0.8, verbose = FALSE)

In [ ]:
seuratObject <- RunUMAP(seuratObject, dims = 1:25, n.neighbors = 12, verbose = FALSE)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(seuratObject, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
# Seurat default settings
seuratObject.markers <- FindAllMarkers(seuratObject, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
seuratObject.markers %>%
    group_by(cluster) %>%
    slice_max(n = 2, order_by = avg_log2FC) 

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

seuratObject.markers[seuratObject.markers$cluster == 0,] %>%
slice_max(n = 30
          , order_by = avg_log2FC)

In [ ]:
write.csv(seuratObject.markers, file = "/home/jovyan/researcher_home/tom/", quote = FALSE)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Col10a1","Ihh","Sp7","Bglap",'Rgs5'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Col10a1","Ihh","Sp7","Bglap",'Col2a1','Pth1r','Prg4','Ucma'), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Runx2","Ogn","Myoc","Mt1","Sp7","Col1a1"), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Scx","Tmnd"), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Col1a1","Fn1",'Col3a1','Col5a2'), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Runx2","Sp7",'Bglap','Ibsp'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Dmp1"), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Postn",'Wnt16','Ctsk','Prrx1'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
#Articular chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Prg4","Col2a1","Sox9"), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
#RZC
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Fgfr3","Pthlh","Ucma","Spon1"), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
#Proliferative chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Mki67","Top2a","Pth1r","Sox9"), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
#Pre-hypertrophic chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Ihh","Ppa1","Smad3"), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
#Hypertrophic chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Col10a1","Mmp13","Ibsp","Sp7"), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
#Periosteal cells
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Fermt3","Ptpn6","Pdgfa"), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
#immune cells
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Pf4","Cd3d","Cd68",'Gp9','Camp','Ccr2'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
#immune cells
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("Cd19","Cd69","Cd4",'Cd8'), 
                min.cutoff = "q1",
                keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 6)
FeaturePlot(seuratObject, features = c('Ihh', 'Col10a1'), blend = TRUE )

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 6)
FeaturePlot(seuratObject, features = c('Col2a1', 'Mki67'), blend = TRUE )

In [ ]:
# Step 1: Define the variable '' with cell type
name <- "Mesenchymal"

# Step 2: Create the column name dynamically
column_name <- paste0("CellType_", name)

In [ ]:
seuratObject@meta.data[[column_name]] <- Idents(seuratObject)

In [ ]:
seuratObject@meta.data[[column_name]] <- as.character(seuratObject@meta.data[[column_name]])

seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '0'] <- 'Subset_osteo'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '1'] <- 'Subset_osteo'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '2'] <- 'Subset_osteo'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '3'] <- 'Subset_osteo'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '4'] <- 'Subset_chondro'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '5'] <- 'Subset_chondro'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '6'] <- 'Periosteal Progenitors'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '7'] <- 'Subset_osteo'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '8'] <- 'Subset_chondro'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '9'] <- 'Periosteal Progenitors'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '10'] <- 'Subset_chondro'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '11'] <- 'Immune cells'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '12'] <- 'Immune cells'

seuratObject@meta.data[[column_name]] <- as.factor(seuratObject@meta.data[[column_name]])

In [ ]:
options(repr.plot.width = 8)

DimPlot(seuratObject, reduction = "umap", label = TRUE, group.by = column_name) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
saveRDS(seuratObject, file = "VSOX_Mutp13_mesenchymal.Rds")

### 2.1.1. Subset chondrocyte

In [ ]:
seuratObject <- readRDS("VSOX_Mutp13_mesenchymal.Rds")

In [ ]:
Idents(seuratObject) <- seuratObject@meta.data$CellType_Mesenchymal

In [ ]:
seuratObject <- subset(seuratObject, idents = c("Subset_chondro"))

In [ ]:
seuratObject <- ScaleData(seuratObject)

In [ ]:
seuratObject <- RunPCA(seuratObject, features = VariableFeatures(object = seuratObject), verbose = FALSE)

In [ ]:
ElbowPlot(seuratObject, 50)

In [ ]:
seuratObject <- FindNeighbors(seuratObject, dims = 1:40, verbose = FALSE)
seuratObject <- FindClusters(seuratObject, resolution = 3.5, verbose = FALSE)

In [ ]:
seuratObject <- RunUMAP(seuratObject, dims = 1:40, n.neighbors = 12, verbose = FALSE)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(seuratObject, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(seuratObject, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
# Seurat default settings
seuratObject.markers <- FindAllMarkers(seuratObject, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
seuratObject.markers %>%
    group_by(cluster) %>%
    slice_max(n = 2, order_by = avg_log2FC) 

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

seuratObject.markers[seuratObject.markers$cluster == 0,] %>%
slice_max(n = 2
          , order_by = avg_log2FC)

In [ ]:
write.csv(seuratObject.markers, file = "/home/jovyan/researcher_home/tom/", quote = FALSE)

In [ ]:
# Chondroprogenitors
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Prrx1", "Prrx2", "Sox9"), order= TRUE, min.cutoff = "q1")  

p1 + p2

In [ ]:
# articular
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Prg4","Col2a1"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Resting zone chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Fgfr3", "Pcp4", "Pthlh", "Sfrp5", "Spon1",'Ucma'), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Proliferative chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Sox9", "Mki67", "Top2a", "Pth1r"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Pre-hypertrophic chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Ihh", "Ppa1", "Smpd3",'Col10a1'), order= TRUE, min.cutoff = "q1")  

p1 + p2

In [ ]:
# SSPC
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Gypa","Car2"), min.cutoff = "q1") 

p1 + p2

In [ ]:
# SSPC
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, c("Sox5","Sox6",'Cd44','Cilp'), min.cutoff = "q1") 

p1 + p2

In [ ]:
# Step 1: Define the variable '' with cell type
name <- "Chondrocyte"

# Step 2: Create the column name dynamically
column_name <- paste0("CellType_", name)

In [ ]:
seuratObject@meta.data[[column_name]] <- Idents(seuratObject)

In [ ]:
seuratObject@meta.data[[column_name]] <- as.character(seuratObject@meta.data[[column_name]])

seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '0'] <- 'Chondroprogenitors'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '1'] <- 'Chondroprogenitors'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '2'] <- 'Resting zone chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '3'] <- 'Resting zone chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '4'] <- 'Hypertrophic chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '5'] <- 'Articular chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '6'] <- 'Pre-hypertrophic chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '7'] <- 'Resting zone chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '8'] <- 'Resting zone chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '9'] <- 'Proliferative chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '10'] <- 'Articular chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '11'] <- 'Resting zone chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '12'] <- 'Proliferative chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '13'] <- 'Proliferative chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '14'] <- 'Chondroprogenitors'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '15'] <- 'Proliferative chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '16'] <- 'Proliferative chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '17'] <- 'Articular chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '18'] <- 'Resting zone chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '19'] <- 'Erythroid cells'

seuratObject@meta.data[[column_name]] <- as.factor(seuratObject@meta.data[[column_name]])

In [ ]:
options(repr.plot.width = 12)

DimPlot(seuratObject, reduction = "umap", label = TRUE, group.by = column_name) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
saveRDS(seuratObject, file = "VSOX_Mutp13_mesenchymal_chondrocyte.Rds")

### 2.1.2. Subset osteo 

In [ ]:
seuratObject <- readRDS("VSOX_Mutp13_mesenchymal.Rds")

In [ ]:
Idents(seuratObject) <- seuratObject@meta.data$CellType_Mesenchymal

In [ ]:
seuratObject <- subset(seuratObject, idents = c("Subset_osteo"))

In [ ]:
seuratObject <- ScaleData(seuratObject)

In [ ]:
seuratObject <- RunPCA(seuratObject, features = VariableFeatures(object = seuratObject), verbose = FALSE)

In [ ]:
ElbowPlot(seuratObject, 50)

In [ ]:
seuratObject <- FindNeighbors(seuratObject, dims = 1:30, verbose = FALSE)
seuratObject <- FindClusters(seuratObject, resolution = 1.5, verbose = FALSE)

In [ ]:
seuratObject <- RunUMAP(seuratObject, dims = 1:30, n.neighbors = 12, verbose = FALSE)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(seuratObject, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
# Seurat default settings
seuratObject.markers <- FindAllMarkers(seuratObject, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
seuratObject.markers %>%
    group_by(cluster) %>%
    slice_max(n = 2, order_by = avg_log2FC) 

In [ ]:
write.csv(seuratObject.markers, file = "/home/jovyan/researcher_home/tom/", quote = FALSE)

In [ ]:
# Osteoprogenitors
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject,
                  c("Sp7","Runx2"),
                    min.cutoff = "q1",
                    keep.scale = 'all',
                    order = TRUE
                     )   

p1 + p2

In [ ]:
# Osteoblast
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject,
                  c("Runx2","Bglap","Lef1","Ibsp"),
                    min.cutoff = "q1",
                    keep.scale = 'all',
                    order = TRUE
                     )   

p1 + p2

In [ ]:
# Osteocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject,
                  c("Dmp1"),
                    min.cutoff = "q1",
                    keep.scale = 'all',
                    order = TRUE
                     )   

p1 + p2

In [ ]:
# Chondro
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject,
                  c("Col10a1","Ihh","Pthlh","Col2a1",'Top2a','Prg4'),
                    min.cutoff = "q1",
                    keep.scale = 'all'
                     )   

p1 + p2

In [ ]:
# Articular chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject,
                  c('Ucma1','Ctsk'),
                    min.cutoff = "q1",
                    #keep.scale = 'all',
                    order = TRUE
                     )   

p1 + p2

In [ ]:
# Articular chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject,
                  c('Ucma1','Ctsk'),
                    min.cutoff = "q1",
                    #keep.scale = 'all',
                    order = TRUE
                     )   

p1 + p2

In [ ]:
# Hyaline chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject,
                  c('Sox5','Sox9','S100a4','Cilp','Cd44','Prrx1','Sp7','Runx2'),
                    min.cutoff = "q1",
                    #keep.scale = 'all',
                    order = TRUE
                     )   

p1 + p2

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(seuratObject, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
# Step 1: Define the variable '' with cell type
name <- "Osteoblast"

# Step 2: Create the column name dynamically
column_name <- paste0("CellType_", name)

In [ ]:
seuratObject@meta.data[[column_name]] <- Idents(seuratObject)

In [ ]:
seuratObject@meta.data[[column_name]] <- as.character(seuratObject@meta.data[[column_name]])

seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '0'] <- 'Chondroprogenitors'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '1'] <- 'Osteoblasts'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '2'] <- 'Articular chondrocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '3'] <- 'Chondroprogenitors'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '4'] <- 'Chondroprogenitors'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '5'] <- 'Osteoprogenitors'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '6'] <- 'Osteocytes'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '7'] <- 'Chondroprogenitors'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '8'] <- 'Osteoblasts'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '9'] <- 'Osteoblasts'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '10'] <- 'Osteoblasts'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '11'] <- 'Osteoblasts'
seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == '12'] <- 'Osteoblasts'

seuratObject@meta.data[[column_name]] <- as.factor(seuratObject@meta.data[[column_name]])

In [ ]:
options(repr.plot.width = 12)

DimPlot(seuratObject, reduction = "umap", label = TRUE, group.by = column_name) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
saveRDS(seuratObject, file = "VSOX_Mutp13_mesenchymal_osteo.Rds")

## 3. Updating annotations

In [ ]:
global <- readRDS("VSOX_Mutp13_mesenchymal.Rds")

In [ ]:
subset1 <- readRDS(file = "VSOX_Mutp13_mesenchymal_chondrocyte.Rds")

In [ ]:
subset2 <- readRDS(file = "VSOX_Mutp13_mesenchymal_osteo.Rds")

In [ ]:
meta1 <- global@meta.data
meta2 <- subset1@meta.data

In [ ]:
df <- split(meta1, meta1$CellType_Mesenchymal)

In [ ]:
subset1a <- rownames(df$Subset_chondro)

In [ ]:
subset1a_meta <- meta2[subset1a, c("CellType_Chondrocyte", "orig.ident")]
subset1a_meta$orig.ident <- NULL

head(subset1a_meta)

In [ ]:
df$Subset_chondro <- cbind(df$Subset_chondro, subset1a_meta)
df$Subset_chondro$CellType_Mesenchymal <- df$Subset_chondro$CellType_Chondrocyte
df$Subset_chondro$CellType_Chondrocyte <- NULL

In [ ]:
meta2 <- subset2@meta.data

In [ ]:
subset2a <- rownames(df$Subset_osteo)

In [ ]:
subset2a_meta <- meta2[subset2a, c("CellType_Osteoblast", "orig.ident")]
subset2a_meta$orig.ident <- NULL

head(subset2a_meta)

In [ ]:
df$Subset_osteo <- cbind(df$Subset_osteo, subset2a_meta)
df$Subset_osteo$CellType_Mesenchymal <- df$Subset_osteo$CellType_Osteoblast
df$Subset_osteo$CellType_Osteoblast <- NULL

In [ ]:
df$`Articular chondrocytes`$Barcode <- rownames(df$`Articular chondrocytes`)
df$`Immune cells`$Barcode <- rownames(df$`Immune cells`)
df$`Proliferative chondrocytes`$Barcode <- rownames(df$`Proliferative chondrocytes`)
df$`Resting zone chondrocytes`$Barcode <- rownames(df$`Resting zone chondrocytes`)
df$`Periosteal Progenitors`$Barcode <- rownames(df$`Periosteal Progenitors`)
df$Osteoblast$Barcode <- rownames(df$Osteoblast)

In [ ]:
df$`Articular chondrocytes`$Barcode = rownames(df$`Articular chondrocytes`)
df$`Immune cells`$Barcode = rownames(df$`Immune cells`)
df$`Proliferative chondrocytes`$Barcode = rownames(df$`Proliferative chondrocytes`)
df$`Resting zone chondrocytes`$Barcode = rownames(df$`Resting zone chondrocytes`)
df$Osteoblast$Barcode = rownames(df$Osteoblast)
df$TBD$Barcode = rownames(df$TBD)
df$Subset_chondro$Barcode = rownames(df$Subset_chondro)
df$Subset_osteo$Barcode = rownames(df$Subset_osteo)

**Make sure all dataframes in the list have the same number of columns, in the same order.**

In [ ]:
str(df)

In [ ]:
names(df)

In [ ]:
names(df)

In [ ]:
meta.data <- do.call(rbind, df)
rownames(meta.data) = meta.data$Barcode
meta.data$Barcode <- NULL

In [ ]:
head(meta.data)

In [ ]:
dim(global@meta.data)
dim(meta.data)

In [ ]:
target <- rownames(global@meta.data)
meta.data <- meta.data[match(target, rownames(meta.data)),]

In [ ]:
global@meta.data = meta.data

In [ ]:
global$CellType <- global$CellType_Mesenchymal
global$CellType_Mesenchymal <- NULL

In [ ]:
seuratObject <- global
rm(global)

#### 5.4.3. Finalized embedding and annotation<a id="37"></a>

In [ ]:
options(repr.plot.width=14)
DimPlot(seuratObject, group.by = "CellType", label = FALSE) + xlab("UMAP 1") + ylab("UMAP 2")

#### 5.4.3. Finalized embedding and annotation<a id="37"></a>

In [ ]:
saveRDS(seuratObject, file = "VSOX_Mutp13_mesenchymal_final.rds")

## 3. Updating annotations

In [ ]:
global <- readRDS("VSOX_Mutp13_global.Rds")

In [ ]:
subset1 <- readRDS(file = "VSOX_Mutp13_mesenchymal_final.rds")

In [ ]:
meta1 <- global@meta.data
meta2 <- subset1@meta.data

In [ ]:
df <- split(meta1, meta1$CellType_Global)

In [ ]:
subset1a <- rownames(df$Mes_lin)

In [ ]:
subset1a_meta <- meta2[subset1a, c("CellType", "orig.ident")]
subset1a_meta$orig.ident <- NULL

head(subset1a_meta)

In [ ]:
df$Mes_lin <- cbind(df$Mes_lin, subset1a_meta)
df$Mes_lin$CellType_Global <- df$Mes_lin$CellType
df$Mes_lin$CellType <- NULL

In [ ]:
df$`Erythroid cells`$Barcode <- rownames(df$`Erythroid cells`)
df$`Immune cells`$Barcode <- rownames(df$`Immune cells`)
df$`Vascular endothelial cells`$Barcode <- rownames(df$`Vascular endothelial cells`)
df$TBD$Barcode <- rownames(df$TBD)
df$Tenocytes$Barcode <- rownames(df$Tenocytes)

In [ ]:
df$`Erythroid cells`$Barcode = rownames(df$`Erythroid cells`)
df$`Immune cells`$Barcode = rownames(df$`Immune cells`)
df$`Vascular endothelial cells`$Barcode = rownames(df$`Vascular endothelial cells`)
df$TBD$Barcode = rownames(df$TBD)
df$Tenocytes$Barcode = rownames(df$Tenocytes)
df$Mes_lin$Barcode = rownames(df$Mes_lin)

**Make sure all dataframes in the list have the same number of columns, in the same order.**

In [ ]:
str(df)

In [ ]:
names(df)

In [ ]:
names(df)

In [ ]:
meta.data <- do.call(rbind, df)
rownames(meta.data) = meta.data$Barcode
meta.data$Barcode <- NULL

In [ ]:
head(meta.data)

In [ ]:
dim(global@meta.data)
dim(meta.data)

In [ ]:
target <- rownames(global@meta.data)
meta.data <- meta.data[match(target, rownames(meta.data)),]

In [ ]:
global@meta.data = meta.data

In [ ]:
global$CellType <- global$CellType_Global
global$CellType_Global <- NULL

In [ ]:
seuratObject <- global
rm(global)

#### 5.4.3. Finalized embedding and annotation<a id="37"></a>

In [ ]:
options(repr.plot.width=14)
DimPlot(seuratObject, group.by = "CellType", label = FALSE) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
saveRDS(seuratObject, file = "VSOX_Mutp13_final.rds")

## Some code

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("","","",""), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
DotPlot(object = seuratObject, features = c('','','',''), col.min = 0)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 6)
FeaturePlot(A, features = c('', ''), blend = TRUE )

### 2.X. [change cell type]

In [ ]:
seuratObject <- readRDS("/home/jovyan/researcher_home/tom/")

In [ ]:
Idents(seuratObject) <- seuratObject@meta.data$CellType_Global

In [ ]:
seuratObject <- subset(seuratObject, idents = c(""))

In [ ]:
seuratObject <- ScaleData(seuratObject)

In [ ]:
seuratObject <- RunPCA(seuratObject, features = VariableFeatures(object = seuratObject), verbose = FALSE)

In [ ]:
ElbowPlot(seuratObject, 50)

In [ ]:
seuratObject <- FindNeighbors(seuratObject, dims = 1:20, verbose = FALSE)
seuratObject <- FindClusters(seuratObject, resolution = 0.8, verbose = FALSE)

In [ ]:
seuratObject <- RunUMAP(seuratObject, dims = 1:20, n.neighbors = 12, verbose = FALSE)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(seuratObject, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
# Seurat default settings
seuratObject.markers <- FindAllMarkers(seuratObject, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

seuratObject.markers[seuratObject.markers$cluster == 0,] %>%
slice_max(n = 20
          , order_by = avg_log2FC)

In [ ]:
write.csv(seuratObject.markers, file = "/home/jovyan/researcher_home/tom/", quote = FALSE)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("","","",""), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(seuratObject, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(seuratObject, 
                c("","","",""), 
                min.cutoff = "q1",
                #keep.scale = 'all',
                order = TRUE
                 )   

p1 + p2

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(seuratObject, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
# Step 1: Define the variable '' with cell type
name <- ""

# Step 2: Create the column name dynamically
column_name <- paste0("CellType_", name)

In [ ]:
seuratObject@meta.data[[column_name]] <- Idents(seuratObject)

In [ ]:
seuratObject@meta.data[[column_name]] <- as.character(seuratObject@meta.data[[column_name]])

seuratObject@meta.data[[column_name]][seuratObject@meta.data[[column_name]] == ''] <- ''

seuratObject@meta.data[[column_name]] <- as.factor(seuratObject@meta.data[[column_name]])

In [ ]:
options(repr.plot.width = 8)

DimPlot(seuratObject, reduction = "umap", label = TRUE, group.by = column_name) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
saveRDS(seuratObject, file = "/home/jovyan/researcher_home/tom/")